In [1]:
from yolov3 import make_seq_yolov3_model
from utils.utils import WeightReader
from utils.utils import preprocess_input
from utils.utils import decode_netout
from utils.utils import correct_yolo_boxes
from utils.utils import do_nms
from utils.bbox import draw_boxes
from utils.utils import get_yolo_boxes
from keras.models import Sequential
from keras.layers import  Dense
import time
import keras.backend as K
import numpy as np
import cv2
import os

Using TensorFlow backend.


In [4]:
net_h, net_w = 416, 416
obj_thresh, nms_thresh = 0.5, 0.45
anchors = [10, 13, 16, 30, 33, 23, 30, 61, 62, 45, 59, 119, 116, 90, 156, 198, 373, 326]
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", \
          "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", \
          "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", \
          "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", \
          "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", \
          "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", \
          "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", \
          "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", \
          "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", \
          "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

imgs_folder_path = 'Z:/dataset/KNU-Campus Dataset/images/20180312_181409/'
img_name = '20180312_181409_'
infer_model,seq_infer_model = make_seq_yolov3_model(input_shape=(416,416,3))

# output_layer_weights = [infer_model.get_layer(name='conv_81').get_weights()[0],
#                         infer_model.get_layer(name='conv_93').get_weights()[0],
#                         infer_model.get_layer(name='conv_105').get_weights()[0]]

In [3]:
def grad_accent(process_img,rows,cols,bs,class_nums,size_nums):
    l_r = 0.0001
    outputs=0
    count=0
    for i in range(len(rows)):
        row = rows[i]
        col = cols[i]
        b = bs[i]
        class_num = class_nums[i]
        size_num = size_nums[i]
        outputs += infer_model.output[yolo_num][:,row,col,20*b +4] + infer_model.output[size_num][:,row,col,20*b +5+ class_num]
        count += 1
    outputs/=count
    start_time = time.time()
    grads = K.gradients(outputs, infer_model.input)[0]
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
    iterate = K.function([infer_model.input], [grads])

    grads_val = iterate([process_img])
    process_img = l_r*grads_val+process_img
    return process_img

In [5]:
if not (os.path.exists('Weights/yolov3.h5')):
    weight_reader = WeightReader('yolov3.weights')
    weight_reader.load_weights(infer_model)
    infer_model.save_weights('Weights/yolov3.h5')
else :
    infer_model.load_weights('Weights/yolov3.h5',by_name=True)
    seq_infer_model.load_weights('Weights/yolov3.h5', by_name=True)

In [ ]:
grads = [[[None]*3]*13]*13
iterates = [[[None]*3]*13]*13

start_time = time.time()
for i in range(1):
    for j in range(1):
        for b in range(3):
            output = infer_model.output[0]#[:,i,j,85*b+4] + infer_model.output[2][:,i,j,85*b+5]
            grad = K.gradients(output, infer_model.input)[0]
            #grad /= (K.sqrt(K.mean(K.square(grad))) + 1e-5)
            iterates[i][j][b] = K.function([infer_model.input], [grad])
print(time.time() - start_time)
# 3개 = 70Mb, 12s
# 425,880 Mb
# 6 * 13*13 + 6 * 26 * 26 + 6 * 52 * 52

# l_r = 0.001
# outputs=0
# count=0
# for i in range(len(rows)):
#     row = rows[i]
#     col = cols[i]
#     b = bs[i]
#     class_num = class_nums[i]
#     yolo_num = yolo_nums[i]
#     outputs += infer_model.output[yolo_num][:,row,col,85*b +4] + infer_model.output[yolo_num][:,row,col,85*b +5+ class_num]
#     count += 1
# outputs/=count
# start_time = time.time()
# grads = K.gradients(outputs, infer_model.input)[0]
# grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
# print(img_num)
# print('grad time',start_time-time.time())
# iterate = K.function([infer_model.input], [grads])
# print('grad time',start_time-time.time())
# grads_val = iterate([process_image])
# print('grad time',start_time-time.time())
# process_image = l_r * grads_val[0]+process_image
# print('grad time',start_time-time.time())
# yolos = seq_infer_model.predict([process_image,prev_feature[0],prev_feature[1],prev_feature[2]])
# prev_feature=yolos[3:]
# yolos=yolos[:3]

In [ ]:
img = cv2.imread(imgs_folder_path+img_name+'0002.jpg')
image_h, image_w, _ = img.shape

process_image = preprocess_input(img, net_h, net_w)



print(time.time() - start_time)

In [6]:
prev_feature=[]
for img_num in range(1000):
    img_num2str=str(img_num)
    while(len(img_num2str)<4):
        img_num2str='0'+img_num2str

    img = cv2.imread(imgs_folder_path+img_name+img_num2str+'.jpg')
    image_h, image_w, _ = img.shape
    
    process_image = preprocess_input(img, net_h, net_w)
    if img_num == 0:
        yolos = infer_model.predict(process_image)
        prev_feature=yolos[3:]
        yolos=yolos[:3]
    else:
        yolos = infer_model.predict(process_image)
        prev_feature=yolos[3:]
        yolos=yolos[:3]
        l_r = 0.001
        outputs=0
        count=0
        for i in range(len(rows)):
            row = rows[i]
            col = cols[i]
            b = bs[i]
            class_num = class_nums[i]
            yolo_num = yolo_nums[i]
            outputs += infer_model.output[yolo_num][:,row,col,85*b +4] + infer_model.output[yolo_num][:,row,col,85*b +5+ class_num]
            count += 1
        outputs/=count
        start_time = time.time()
        grads = K.gradients(outputs, infer_model.input)[0]
        grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
        print(img_num)
        print('grad time',start_time-time.time())
        iterate = K.function([infer_model.input], [grads])
        print('grad time',start_time-time.time())
        grads_val = iterate([process_image])
        print('grad time',start_time-time.time())
        process_image = l_r * grads_val[0]+process_image
        print('grad time',start_time-time.time())
        yolos = seq_infer_model.predict([process_image,prev_feature[0],prev_feature[1],prev_feature[2]])
        prev_feature=yolos[3:]
        yolos=yolos[:3]
        
        
    
    #yolos = infer_model.predict(process_image)
    boxes = []

    for i in range(len(yolos)):
        # decode the output of the network
        yolo_anchors = anchors[(2 - i) * 6:(3 - i) * 6]
        boxes += decode_netout(yolos[i][0], yolo_anchors, obj_thresh, net_h, net_w, i)
    #print(boxes)
    # correct the sizes of the bounding boxes
    correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w)

    # suppress non-maximal boxes
    do_nms(boxes, nms_thresh)

    # draw bounding boxes on the image using labels
    rows,cols,bs,class_nums,yolo_nums = draw_boxes(img, boxes, labels, obj_thresh, debug = False)
    #cv2.imwrite('outputs/detect/'+img_name+img_num2str+'_detected.jpg',cv2.resize(img,(1280,720)))
    cv2.imshow('video with bboxes', cv2.resize(img,(1280,720)))
    cv2.waitKey(6)
#     print()

1
grad time -3.561194658279419
grad time -3.5616955757141113
grad time -7.479043245315552
grad time -7.482050180435181
2
grad time -3.5069143772125244
grad time -3.523475170135498
grad time -6.921703577041626
grad time -6.925214052200317
3
grad time -3.2562172412872314
grad time -3.272758960723877
grad time -6.775151014328003
grad time -6.778159141540527
4
grad time -3.507351875305176
grad time -3.5234227180480957
grad time -7.067072629928589
grad time -7.070581912994385
5
grad time -3.3253965377807617
grad time -3.3414156436920166
grad time -7.005884885787964
grad time -7.008892774581909
6
grad time -3.465247392654419
grad time -3.481292724609375
grad time -7.29293155670166
grad time -7.296441078186035


KeyboardInterrupt: 

In [ ]:
infer_model.output[0]

In [ ]:
cap = cv2.VideoCapture('Z:/dataset/KNU-Campus Dataset/videos/scene11.mp4')

In [ ]:
count=0
while(True):
    count2str = str(count)
    
    while(len(count2str)<4):
        count2str='0'+count2str
    ret,frame = cap.read()
    if (ret):
        cv2.imwrite(imgs_folder_path+img_name+count2str+'.jpg',frame)
        count+=1